## Decision tree of fruits 
It's a decision tree model that shows what question ask to find each fruits <br>
Method **print_tree** print qestions and fruits that mach to those questions eg: <br> <br>

###################<br>
<pre>
Is size >= 2?
--> True:
  Predict {'Apple': 2}
--> False: 
  Predict {'Grape': 1}
</pre>
####################<br>

<br>
Below data is predicted with 100% of accuracy for each fruit. And model gives what question ask to distinguish each fruit

In [20]:
# Traingin data

training_data = [
    ['Green',   10, "Round", 'Mango'], 
    ['Yellow',  12, "Round", 'Mango'],
    ['Red',      7, "Round", 'Apple'],
    ['Violet', 0.6, "Round", 'Grape'],
    ['Green',  0.8, "Round", 'Grape'],
    ['Yellow',   5, "Round", 'Lemon'],
    ['Red',      2, "Round", 'Cherry'],
    ['Yellow',  17,  "Long", 'Banana'],
]

header = ["color", "size", "shape", "label"]


In [2]:
def unique_values(rows, col):
    return set([row[col] for row in rows])


In [3]:
def class_counts(rows):
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [4]:
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

In [5]:
class Question:
    def __init__(self, column, value):
        self.column = column
        self.value = value
    
    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
        
    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (header[self.column], condition, str(self.value))
    


In [6]:
def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows


In [7]:

def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity


In [8]:

def info_gain(left, right, current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p *gini(left) - (1-p) * gini(right)

In [9]:
def find_best_split(rows):
    best_gain = 0 # keep the best information gain
    best_question = None
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1
    
    for col in range(n_features):
        values = set([row[col] for row in rows])
        
        for val in values:
            question = Question(col, val)
            true_rows, false_rows = partition(rows, question)
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
                
            gain = info_gain(true_rows, false_rows, current_uncertainty)
            if gain >= best_gain:
                best_gain, best_question = gain, question
                
    return best_gain, best_question

In [10]:
class Leaf:
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [11]:
class Decision_Node:
    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch
        

In [12]:
def build_tree(rows):
    gain, question = find_best_split(rows)
    if gain == 0:
        return Leaf(rows)
    
    true_rows, false_rows = partition(rows, question)
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    
    return Decision_Node(question, true_branch, false_branch)

In [13]:
def print_tree(node, spacing=""):
    if isinstance(node, Leaf):
        print(spacing + "Predict", node.predictions)
        return
    
    print(spacing + str(node.question))
    
    print(spacing + "--> True:")
    print_tree(node.true_branch, spacing + "  ")
    
    print(spacing + "--> False: ")
    print_tree(node.false_branch, spacing + "  ")

    

In [14]:
def classify(row, node):
    if isinstance(node, Leaf):
        return node.predictions
    
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [15]:
def print_leaf(counts):
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [16]:
###########

In [17]:
testing_data = [
    ['Green',   11, "Round", 'Mango'], 
    ['Red',      8, "Round", 'Apple'],
    ['Violet', 0.6, "Round", 'Grape'],
    ['Green',  0.8, "Round", 'Grape'],
    ['Yellow',   5, "Round", 'Lemon'],
    ['Red',      2, "Round", 'Cherry'],
    ['Yellow',  17.5,  "Long", 'Banana'],
]

my_tree = build_tree(training_data)
print_tree(my_tree)

for row in testing_data:
    print("Actual: %s. Predicted: %s" % (row[-1], print_leaf(classify(row, my_tree))))
    

Is size >= 2?
--> True:
  Is size >= 10?
  --> True:
    Is shape == Round?
    --> True:
      Predict {'Mango': 2}
    --> False: 
      Predict {'Banana': 1}
  --> False: 
    Is size >= 5?
    --> True:
      Is size >= 7?
      --> True:
        Predict {'Apple': 1}
      --> False: 
        Predict {'Lemon': 1}
    --> False: 
      Predict {'Cherry': 1}
--> False: 
  Predict {'Grape': 2}
Actual: Mango. Predicted: {'Mango': '100%'}
Actual: Apple. Predicted: {'Apple': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Lemon. Predicted: {'Lemon': '100%'}
Actual: Cherry. Predicted: {'Cherry': '100%'}
Actual: Banana. Predicted: {'Banana': '100%'}


## Summary
As you can see in the result each fruit has 100% of accuracy. Also model gave us the best question we can ask to quickly distinguish each fruit based on its feature.